In [402]:
import pandas as pd
import numpy as np

In [403]:
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Ram\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.4                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following packages will be UPDATED:

  conda                                4.8.4-py37hc8dfbb8_0 --> 4.8.4-py37hc8dfbb8_1




conda-4.8.4          | 3.1 MB    |            |   0% 
conda-4.8.4          | 3.1 MB    |            |   1% 
conda-4.8.4          | 3.1 MB    | #####4     |  55% 
conda-4.8.4          | 3.1 MB    | ########## | 100% 
conda-4.8.4          | 3.1 MB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing trans

In [404]:
import requests

Using this website to generate all the zipcodes in Suffolk County (home of Boston)

In [405]:
url = 'https://www.zip-codes.com/city/ma-boston.asp'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[2]
print(df)
df.to_csv('Boston_postal.csv')

                 0         1        2           3  \
0         ZIP Code      Type   County  Population   
1   ZIP Code 02108  Standard  Suffolk        3825   
2   ZIP Code 02109  Standard  Suffolk        3771   
3   ZIP Code 02110  Standard  Suffolk        1733   
4   ZIP Code 02111  Standard  Suffolk        7383   
5   ZIP Code 02112  P.O. Box  Suffolk           0   
6   ZIP Code 02113  Standard  Suffolk        6915   
7   ZIP Code 02114  Standard  Suffolk       11999   
8   ZIP Code 02115  Standard  Suffolk       28441   
9   ZIP Code 02116  Standard  Suffolk       20628   
10  ZIP Code 02117  P.O. Box  Suffolk           0   
11  ZIP Code 02118  Standard  Suffolk       26498   
12  ZIP Code 02119  Standard  Suffolk       25346   
13  ZIP Code 02120  Standard  Suffolk       15181   
14  ZIP Code 02121  Standard  Suffolk       25978   
15  ZIP Code 02122  Standard  Suffolk       23479   
16  ZIP Code 02123  P.O. Box  Suffolk           0   
17  ZIP Code 02124  Standard  Suffolk       47

In [406]:
newDF = df.drop(columns = [1,3,4])

In [407]:
newDF.columns = newDF.iloc[0]

In [408]:
newDF = newDF.drop([0,0])

In [409]:
newDF['ZIP Code'] = newDF['ZIP Code'].str[9:]

In [410]:
newDF["ZIP Code"] = newDF["ZIP Code"].astype(str).astype(int)

In [411]:
filename = 'US_Zip_Codes.csv'
df2 = pd.read_csv('US_Zip_Codes.csv')

In [412]:
df2 = df2.rename(columns={"ZIP": "ZIP Code"})

In [413]:
df2.head()

,ZIP Code,LAT,LNG
0,601,18.180555,-66.749961
1,602,18.361945,-67.175597
2,603,18.455183,-67.119887
3,606,18.158345,-66.932911
4,610,18.295366,-67.125135


In [414]:
fullDF = newDF.merge(df2, on='ZIP Code', how='left')

In [415]:
nonNA = fullDF.dropna()

In [416]:
nonNA.head()

,ZIP Code,County,LAT,LNG
0,2108,Suffolk,42.357768,-71.064858
1,2109,Suffolk,42.367032,-71.050493
2,2110,Suffolk,42.361962,-71.047846
3,2111,Suffolk,42.350518,-71.059077
5,2113,Suffolk,42.365331,-71.055233


In [417]:
df3 = pd.read_csv('neighborhood_data.csv')

In [418]:
df3 = df3.rename(columns={"Zip Code": "ZIP Code"})
df3.head()

,OBJECTID,Name,Acres,Neighborhood_ID,SqMiles,ShapeSTArea,ShapeSTLength,ZIP Code
0,27,Roslindale,1605.568238,15,2.51,6.993827e+07,53563.912600,2131
1,28,Jamaica Plain,2519.245394,11,3.94,1.097379e+08,56349.937160,2130
2,29,Mission Hill,350.853564,13,0.55,1.528312e+07,17918.724110,2120
3,30,Longwood,188.611947,28,0.29,8.215904e+06,11908.757150,2115
4,31,Bay Village,26.539839,33,0.04,1.156071e+06,4650.635493,2116


In [419]:
df3.dtypes

OBJECTID             int64
Name                object
Acres              float64
Neighborhood_ID      int64
SqMiles            float64
ShapeSTArea        float64
ShapeSTLength      float64
ZIP Code             int64
dtype: object

In [420]:
fullDF2 = fullDF.merge(df3, on='ZIP Code', how='left')

In [421]:
fullDF2.head()

,ZIP Code,County,LAT,LNG,OBJECTID,Name,Acres,Neighborhood_ID,SqMiles,ShapeSTArea,ShapeSTLength
0,2108,Suffolk,42.357768,-71.064858,41.0,Beacon Hill,200.156904,30.0,0.31,8.718800e+06,14303.829020
1,2109,Suffolk,42.367032,-71.050493,52.0,Harbor Islands,824.888658,22.0,1.29,3.593201e+07,92482.183570
2,2110,Suffolk,42.361962,-71.047846,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2111,Suffolk,42.350518,-71.059077,32.0,Leather District,15.639908,27.0,0.02,6.812717e+05,3237.140537
4,2111,Suffolk,42.350518,-71.059077,33.0,Chinatown,76.324410,26.0,0.12,3.324678e+06,9736.590413


In [422]:
#fullDF2 = fullDF2.drop(columns=['OBJECTID', 'County', 'Neighborhood_ID'])
fullDF2 = fullDF2.drop(columns=['Neighborhood_ID'])

In [423]:
finalDF = fullDF2.dropna()

In [424]:
finalDF.reset_index(drop=True, inplace=True)

In [425]:
finalDF.head()

,ZIP Code,County,LAT,LNG,OBJECTID,Name,Acres,SqMiles,ShapeSTArea,ShapeSTLength
0,2108,Suffolk,42.357768,-71.064858,41.0,Beacon Hill,200.156904,0.31,8.718800e+06,14303.829020
1,2109,Suffolk,42.367032,-71.050493,52.0,Harbor Islands,824.888658,1.29,3.593201e+07,92482.183570
2,2111,Suffolk,42.350518,-71.059077,32.0,Leather District,15.639908,0.02,6.812717e+05,3237.140537
3,2111,Suffolk,42.350518,-71.059077,33.0,Chinatown,76.324410,0.12,3.324678e+06,9736.590413
4,2113,Suffolk,42.365331,-71.055233,34.0,North End,126.910439,0.20,5.527506e+06,16177.826820


Finally, the required neighborhood dataframes have been assembled properly. For this project, I am following the similar concept from the previous project from week 3 where I would use the following values: latitude, longitude, and for identifying purposes, the name of the neighborhood. In case it was difficult to tell, I initially parsed through a list of zip codes in Boston from a website, and then I combined that list of zip codes with a csv file that had the corresponding latitude and longitude with the Zip code being the key. I found this file from a github: https://gist.github.com/erichurst/7882666. 
Once I had this, I combined this with a csv file of the Boston neighborhoods based on Zip codes (which I created myself), and used the Zip as the key once again. This was going to be my final dataframe for the project (finalDF). 

**Creating Map of Boston**

In [426]:
address = 'Boston, MA'

geolocator = Nominatim(user_agent="Boston_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Boston are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Boston are 42.3602534, -71.0582912.


In [427]:
# Creating a map of Boston
map_Boston = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(finalDF['LAT'], finalDF['LNG'], finalDF["Name"]):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Boston)  
    
map_Boston

As you can see from the map produced, it only displays 19 of the 25 neighborhoods. This is because the neighborhoods from the csv files repeat the same latitude and longitudes for the location since some of them share the same zip codes. As a result, the map relied on the larger area defined neighborhoods to display their names instead. 

In [428]:
CLIENT_ID = 'LQSN2LMHVZWTW4Y50YZSPI3QGXFUWWZARIWV213WFE2XJBB4' # your Foursquare ID
CLIENT_SECRET = 'RRD2SQQNAXTRGPRHJN0UX3CA5JNIIBPUGS4PLEK2YI4RNTRH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [429]:
neighborhood_latitude = finalDF.loc[0, 'LAT'] # neighborhood latitude value
neighborhood_longitude = finalDF.loc[0, 'LNG'] # neighborhood longitude value
neighborhood_name = finalDF.loc[0, 'Name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Beacon Hill are 42.357768, -71.064858.


In [430]:
LIMIT = 100
radius = 800
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LQSN2LMHVZWTW4Y50YZSPI3QGXFUWWZARIWV213WFE2XJBB4&client_secret=RRD2SQQNAXTRGPRHJN0UX3CA5JNIIBPUGS4PLEK2YI4RNTRH&v=20180604&ll=42.357768,-71.064858&radius=800&limit=100'

In [431]:
results = requests.get(url).json()
#results

In [432]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [433]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()
#nearby_venues

C:\Users\Ram\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Boston Common,Park,42.355487,-71.064882
1,Boston Athenaeum,Library,42.357481,-71.061838
2,Frog Pond,Lake,42.356134,-71.065672
3,Florina,Pizza Place,42.359375,-71.063219
4,No. 9 Park,French Restaurant,42.357540,-71.063193


In [434]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [435]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [436]:
Boston_venues = getNearbyVenues(names=finalDF['Name'], latitudes=finalDF['LAT'], longitudes=finalDF['LNG'])

Beacon Hill
Harbor Islands
Leather District
Chinatown
North End
West End
Longwood
Back Bay
Fenway
Bay Village
Roxbury
Mission Hill
Dorchester
Mattapan
South Boston Waterfront
South Boston
East Boston
Charlestown
Jamaica Plain
Roslindale
South End
West Roxbury
Allston
Brighton
Hyde Park


In [437]:
print(Boston_venues.shape)
Boston_venues.tail(5)

(976, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
971,Hyde Park,42.255083,-71.12922,SUBWAY,42.255269,-71.124482,Sandwich Place
972,Hyde Park,42.255083,-71.12922,Citizens Bank Supermarket Branch,42.251956,-71.125644,Bank
973,Hyde Park,42.255083,-71.12922,Rite Aid,42.253319,-71.125626,Pharmacy
974,Hyde Park,42.255083,-71.12922,Shaw's,42.251787,-71.126090,Grocery Store
975,Hyde Park,42.255083,-71.12922,McDonald's,42.252866,-71.124458,Fast Food Restaurant


In [438]:
Boston_venues.head(5)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Beacon Hill,42.357768,-71.064858,Frog Pond,42.356134,-71.065672,Lake
1,Beacon Hill,42.357768,-71.064858,Boston Common,42.355487,-71.064882,Park
2,Beacon Hill,42.357768,-71.064858,Boston Athenaeum,42.357481,-71.061838,Library
3,Beacon Hill,42.357768,-71.064858,No. 9 Park,42.357540,-71.063193,French Restaurant
4,Beacon Hill,42.357768,-71.064858,Florina,42.359375,-71.063219,Pizza Place


In [439]:
Boston_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(Boston_venues['Venue Category'].unique())))

There are 184 uniques categories.


In [440]:
Boston_onehot = pd.get_dummies(Boston_venues[['Venue Category']], prefix="", prefix_sep="")
Boston_onehot.drop(['Neighborhood'], axis=1, inplace = True)
Boston_onehot.insert(loc=0, column='Neighborhood', value=Boston_venues['Neighborhood'] )
Boston_onehot.shape

(976, 184)

In [441]:
Boston_grouped = Boston_onehot.groupby('Neighborhood').mean().reset_index()
Boston_grouped.shape

(25, 184)

In [442]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [443]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Boston_grouped['Neighborhood']

for ind in np.arange(Boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Boston_grouped.iloc[ind, :], num_top_venues)

    #neighborhoods_venues_sorted.head(25)

In [444]:
# Getting the most common venues for each neighborhood
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,Rental Car Location,Thrift / Vintage Store,Record Shop,Pizza Place,Smoke Shop,Nightclub,Chinese Restaurant,Liquor Store,Donut Shop,Bar
1,Back Bay,Café,Coffee Shop,Falafel Restaurant,Sushi Restaurant,Sandwich Place,Pub,Bar,Fast Food Restaurant,Shipping Store,Pharmacy
2,Bay Village,American Restaurant,Coffee Shop,Hotel,Spa,Seafood Restaurant,Clothing Store,Gym,Department Store,Gym / Fitness Center,Mediterranean Restaurant
3,Beacon Hill,Coffee Shop,American Restaurant,Pizza Place,Italian Restaurant,French Restaurant,Sushi Restaurant,Museum,Restaurant,New American Restaurant,Plaza
4,Brighton,Bus Station,Bank,Bakery,Pizza Place,Dry Cleaner,Chinese Restaurant,Coffee Shop,Café,Metro Station,Salon / Barbershop


Since there are a total of 185 unique categories, trying to identify each and every single venue to assign it a weight would be less than ideal. As a result, I am going to be using the 10 most common venues for each neighborhood. Even still, each neighborhood has a different list of 10 venues, so I will have to use key words to group certain places as one. For example, all venues with the word restaurant will be treated the same- umbrella under restaurant, fast food stores and pizza places, pubs and bars, etc. This will still retain the important information from each venue, but it will make it easier to group in order to perform ML algorithmic analysis on the data.

In [445]:
neighborhoods_venues_sorted.dtypes

Neighborhood              object
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
4th Most Common Venue     object
5th Most Common Venue     object
6th Most Common Venue     object
7th Most Common Venue     object
8th Most Common Venue     object
9th Most Common Venue     object
10th Most Common Venue    object
dtype: object

In [446]:
# Venue Categories: mall, 
easy = neighborhoods_venues_sorted

In [447]:
easy.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,Rental Car Location,Thrift / Vintage Store,Record Shop,Pizza Place,Smoke Shop,Nightclub,Chinese Restaurant,Liquor Store,Donut Shop,Bar
1,Back Bay,Café,Coffee Shop,Falafel Restaurant,Sushi Restaurant,Sandwich Place,Pub,Bar,Fast Food Restaurant,Shipping Store,Pharmacy
2,Bay Village,American Restaurant,Coffee Shop,Hotel,Spa,Seafood Restaurant,Clothing Store,Gym,Department Store,Gym / Fitness Center,Mediterranean Restaurant
3,Beacon Hill,Coffee Shop,American Restaurant,Pizza Place,Italian Restaurant,French Restaurant,Sushi Restaurant,Museum,Restaurant,New American Restaurant,Plaza
4,Brighton,Bus Station,Bank,Bakery,Pizza Place,Dry Cleaner,Chinese Restaurant,Coffee Shop,Café,Metro Station,Salon / Barbershop


In [448]:
#df.loc[(df.Event == 'Dance'),'Event']='Hip-Hop'
#easy['1st Most Common Venue'].replace(['Restaurant'], 'Restaurant')

I was going to attempt to try to convert the objects into strings and make a more comprehensive model, but it was too difficult, and the deadline was approaching. Going to get back to this section :).

In [449]:
#easy.loc[easy.str.contains('Restaurant')] = 'Restaurant'

In [450]:
#for i, row in easy.iterrows():
#    ifor_val = something
#    if (row[i]):
#        ifor_val = something_else
#    df.set_value(i,'ifor',ifor_val)

In [451]:
easy.dtypes

Neighborhood              object
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
4th Most Common Venue     object
5th Most Common Venue     object
6th Most Common Venue     object
7th Most Common Venue     object
8th Most Common Venue     object
9th Most Common Venue     object
10th Most Common Venue    object
dtype: object

In [468]:
Boston_grouped.head()

,Neighborhood,Accessories Store,African Restaurant,Airport Service,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,...,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Allston,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0
1,Back Bay,0.0,0.0,0.0,0.032258,0.0,0.00,0.000000,0.000000,0.0,...,0.0,0.032258,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0
2,Bay Village,0.0,0.0,0.0,0.060000,0.0,0.01,0.000000,0.010000,0.0,...,0.0,0.010000,0.000000,0.0,0.0,0.020000,0.02,0.000000,0.0,0.0
3,Beacon Hill,0.0,0.0,0.0,0.039474,0.0,0.00,0.013158,0.013158,0.0,...,0.0,0.000000,0.013158,0.0,0.0,0.000000,0.00,0.013158,0.0,0.0
4,Brighton,0.0,0.0,0.0,0.022222,0.0,0.00,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.022222,0.00,0.000000,0.0,0.0


In [ ]:
kclusters = 5
Boston_clustering = Boston_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_.astype('int32') 

**Clustering Neighborhoods** 

In [500]:
# set number of clusters
kclusters = 5
Boston_clustering = Boston_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, n_init=10, random_state=0, max_iter=15).fit(Boston_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_.astype('int32') 

array([2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 4, 2, 0, 2, 3, 2, 2,
       2, 2, 2])

In [501]:
kmeans.labels_.astype('int32')

array([2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 4, 2, 0, 2, 3, 2, 2,
       2, 2, 2])

In [502]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [503]:
Boston_merged = finalDF
Boston_merged = Boston_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Name')
#Boston_merged # check the last columns!

In [504]:
Boston_merged.head(5)

,ZIP Code,County,LAT,LNG,OBJECTID,Name,Acres,SqMiles,ShapeSTArea,ShapeSTLength,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2108,Suffolk,42.357768,-71.064858,41.0,Beacon Hill,200.156904,0.31,8.718800e+06,14303.829020,...,Coffee Shop,American Restaurant,Pizza Place,Italian Restaurant,French Restaurant,Sushi Restaurant,Museum,Restaurant,New American Restaurant,Plaza
1,2109,Suffolk,42.367032,-71.050493,52.0,Harbor Islands,824.888658,1.29,3.593201e+07,92482.183570,...,Italian Restaurant,Pizza Place,Harbor / Marina,Park,Seafood Restaurant,Café,Pastry Shop,Outdoor Sculpture,Church,Diner
2,2111,Suffolk,42.350518,-71.059077,32.0,Leather District,15.639908,0.02,6.812717e+05,3237.140537,...,Asian Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Sushi Restaurant,Sandwich Place,Theater,Café,Gym / Fitness Center,Restaurant
3,2111,Suffolk,42.350518,-71.059077,33.0,Chinatown,76.324410,0.12,3.324678e+06,9736.590413,...,Asian Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Sushi Restaurant,Sandwich Place,Theater,Café,Gym / Fitness Center,Restaurant
4,2113,Suffolk,42.365331,-71.055233,34.0,North End,126.910439,0.20,5.527506e+06,16177.826820,...,Italian Restaurant,Pizza Place,Seafood Restaurant,Park,Coffee Shop,Café,Sports Bar,Bakery,Pub,Sandwich Place


In [505]:
Boston_merged = Boston_merged.dropna()

In [506]:
Boston_merged = Boston_merged.astype({'Cluster Labels': 'Int64'})
#Boston_merged.head(25)

In [507]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Boston_merged['LAT'], Boston_merged['LNG'], Boston_merged['Name'], Boston_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=1.0).add_to(map_clusters)
       
map_clusters

In [577]:
#Boston_merged.loc[Boston_merged['Cluster Labels'] == 0, Boston_merged.columns[[1] + list(range(5, Boston_merged.shape[1]))]]

In [514]:
FDF = Boston_merged.drop(columns = ['County', 'Acres', 'SqMiles', 'ShapeSTArea', 'ShapeSTLength', 'ZIP Code', 'LAT', 'LNG', 'OBJECTID'])
#displayDF.head()

In [574]:
#FDF.loc[FDF['Cluster Labels'] == 0, FDF.columns[[1] + list(range(2, FDF.shape[1]))]]

In [582]:
FDF.loc[FDF['Cluster Labels'] == 0, FDF.columns[[1] + list(range(2, FDF.shape[1]))]]

,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Score
1,0,Italian Restaurant,Pizza Place,Harbor / Marina,Park,Seafood Restaurant,Café,Pastry Shop,Outdoor Sculpture,Church,Diner,53
4,0,Italian Restaurant,Pizza Place,Seafood Restaurant,Park,Coffee Shop,Café,Sports Bar,Bakery,Pub,Sandwich Place,66


In [584]:
FDF.loc[FDF['Cluster Labels'] == 4, FDF.columns[[1] + list(range(2, FDF.shape[1]))]]

,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Score
13,4,Pizza Place,Donut Shop,Hot Dog Joint,Event Service,Home Service,Park,Zoo Exhibit,Food Truck,Food & Drink Shop,Fast Food Restaurant,49


In [579]:
FDF.loc[FDF['Cluster Labels'] == 2, FDF.columns[[1] + list(range(2, FDF.shape[1]))]]

,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Score
0,2,Coffee Shop,American Restaurant,Pizza Place,Italian Restaurant,French Restaurant,Sushi Restaurant,Museum,Restaurant,New American Restaurant,Plaza,62
2,2,Asian Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Sushi Restaurant,Sandwich Place,Theater,Café,Gym / Fitness Center,Restaurant,64
3,2,Asian Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Sushi Restaurant,Sandwich Place,Theater,Café,Gym / Fitness Center,Restaurant,64
5,2,Pizza Place,Science Museum,Food Truck,Café,Hotel Bar,Museum,American Restaurant,Pharmacy,Donut Shop,Liquor Store,60
6,2,Café,Coffee Shop,Falafel Restaurant,Sushi Restaurant,Sandwich Place,Pub,Bar,Fast Food Restaurant,Shipping Store,Pharmacy,68
7,2,Café,Coffee Shop,Falafel Restaurant,Sushi Restaurant,Sandwich Place,Pub,Bar,Fast Food Restaurant,Shipping Store,Pharmacy,68
8,2,Café,Coffee Shop,Falafel Restaurant,Sushi Restaurant,Sandwich Place,Pub,Bar,Fast Food Restaurant,Shipping Store,Pharmacy,68
9,2,American Restaurant,Coffee Shop,Hotel,Spa,Seafood Restaurant,Clothing Store,Gym,Department Store,Gym / Fitness Center,Mediterranean Restaurant,60
11,2,Pizza Place,Donut Shop,Liquor Store,Track,Burger Joint,New American Restaurant,Sushi Restaurant,Mexican Restaurant,Gym,Furniture / Home Store,57
12,2,Fast Food Restaurant,Zoo Exhibit,Men's Store,Cosmetics Shop,Pharmacy,Donut Shop,Caribbean Restaurant,Shopping Mall,Nightclub,Farmers Market,56


In [571]:
#FDF.loc[FDF['Cluster Labels'] == 3, FDF.columns[[1] + list(range(2, FDF.shape[1]))]]


**Scoring of 10 most common venues (1-10)**

1 = low risk, 10 = high risk

- Restaurant = 7
- Cafe = 6
- Museum/Gallery/Exhibitions = 5
- Bar/Pub = 9
- fitness related = 6
- park/outdoor areas = 2
- Grocery/Convenience store = 6
- Misc store = 4
- Nail salon/barbershops = 8
- Non-close contact service = 3
- Food truck = 5
- Transportation = 5

Hardcoding a score (in order) based on types of most common venues in each neighborhood

In [531]:
score_up = pd.DataFrame({'Score': [62,53,64,64,66,60,68,68,68,60,48,57,56,49,70,70,54,58,60,70,70,52,59,58,63]})

In [534]:
FDF['Score'] = FDF.apply(lambda x: 10, axis = 1)

In [535]:
FDF.update(score_up)

In [546]:
C0 = FDF.loc[FDF['Cluster Labels'] == 0, FDF.columns[[1] + list(range(2, FDF.shape[1]))]]

In [547]:
C1 = FDF.loc[FDF['Cluster Labels'] == 1, FDF.columns[[1] + list(range(2, FDF.shape[1]))]]

In [548]:
C2 = FDF.loc[FDF['Cluster Labels'] == 2, FDF.columns[[1] + list(range(2, FDF.shape[1]))]]

In [549]:
C3 = FDF.loc[FDF['Cluster Labels'] == 3, FDF.columns[[1] + list(range(2, FDF.shape[1]))]]

In [550]:
C4 = FDF.loc[FDF['Cluster Labels'] == 4, FDF.columns[[1] + list(range(2, FDF.shape[1]))]]

In [569]:
print("Cluster 0 average score:",C0['Score'].mean())
print("Cluster 1 average score:",C1['Score'].mean())
print("Cluster 2 average score:",C2['Score'].mean())
print("Cluster 3 average score:",C3['Score'].mean())
print("Cluster 4 average score:",C4['Score'].mean())

Cluster 0 average score: 59.5
Cluster 1 average score: 54.0
Cluster 2 average score: 62.85
Cluster 3 average score: 48.0
Cluster 4 average score: 49.0


**Analysis of Results**


After performing clustering on the neighborhoods, I used a scoring system to rank the neighborhoods in terms of risk. 
After clustering was performed based on the venues in each neighborhood, it arrived at the ideal number within 5 iterations. I, then took the average score for each cluster to identify the risk ratings for each cluster of neighborhoods. 

From my results, the Clusters from lowest risk to highest risk are: 5,4,2,1,3.
This seems to imply that neighborhoods in these areas tend to have a lot of social gathering venues such as bars, restaurants, and fewer outdoor, spaced-out areas such as parks, etc. 


In [578]:
FDF

,Name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Score
0,Beacon Hill,2,Coffee Shop,American Restaurant,Pizza Place,Italian Restaurant,French Restaurant,Sushi Restaurant,Museum,Restaurant,New American Restaurant,Plaza,62
1,Harbor Islands,0,Italian Restaurant,Pizza Place,Harbor / Marina,Park,Seafood Restaurant,Café,Pastry Shop,Outdoor Sculpture,Church,Diner,53
2,Leather District,2,Asian Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Sushi Restaurant,Sandwich Place,Theater,Café,Gym / Fitness Center,Restaurant,64
3,Chinatown,2,Asian Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Sushi Restaurant,Sandwich Place,Theater,Café,Gym / Fitness Center,Restaurant,64
4,North End,0,Italian Restaurant,Pizza Place,Seafood Restaurant,Park,Coffee Shop,Café,Sports Bar,Bakery,Pub,Sandwich Place,66
5,West End,2,Pizza Place,Science Museum,Food Truck,Café,Hotel Bar,Museum,American Restaurant,Pharmacy,Donut Shop,Liquor Store,60
6,Longwood,2,Café,Coffee Shop,Falafel Restaurant,Sushi Restaurant,Sandwich Place,Pub,Bar,Fast Food Restaurant,Shipping Store,Pharmacy,68
7,Back Bay,2,Café,Coffee Shop,Falafel Restaurant,Sushi Restaurant,Sandwich Place,Pub,Bar,Fast Food Restaurant,Shipping Store,Pharmacy,68
8,Fenway,2,Café,Coffee Shop,Falafel Restaurant,Sushi Restaurant,Sandwich Place,Pub,Bar,Fast Food Restaurant,Shipping Store,Pharmacy,68
9,Bay Village,2,American Restaurant,Coffee Shop,Hotel,Spa,Seafood Restaurant,Clothing Store,Gym,Department Store,Gym / Fitness Center,Mediterranean Restaurant,60


48.0

49.0

61.08